In [1]:
# import pandas as pd, numpy as np
# from hpfrec import HPF

# ## Generating sample counts data
# nusers = 15400
# nitems = 1000
# nobs   = 

# np.random.seed(1)
# counts_df = pd.DataFrame({
# 	'UserId' : np.random.randint(nusers, size=nobs),
# 	'ItemId' : np.random.randint(nitems, size=nobs),
# 	'Count' :  (np.random.gamma(1,1, size=nobs) + 1).astype('int32')
# 	})
# counts_df = counts_df.loc[~counts_df[['UserId', 'ItemId']].duplicated()].reset_index(drop=True)

# ## Initializing the model object
# recommender = HPF()

# ## For stochastic variational inference, need to select batch size (number of users)
# recommender = HPF(users_per_batch = 20)

# ## Full function call
# recommender = HPF(
# 	k=30, a=0.3, a_prime=0.3, b_prime=1.0,
# 	c=0.3, c_prime=0.3, d_prime=1.0, ncores=-1,
# 	stop_crit='train-llk', check_every=10, stop_thr=1e-3,
# 	users_per_batch=None, items_per_batch=None, step_size=lambda x: 1/np.sqrt(x+2),
# 	maxiter=100, use_float=True, reindex=True, verbose=True,
# 	random_seed=None, allow_inconsistent_math=False, full_llk=False,
# 	alloc_full_phi=False, keep_data=True, save_folder=None,
# 	produce_dicts=True, keep_all_objs=True, sum_exp_trick=False
# )

# ## Fitting the model to the data
# recommender.fit(counts_df)

# ## Fitting the model while monitoring a validation set
# recommender = HPF(stop_crit='val-llk')
# recommender.fit(counts_df, val_set=counts_df.sample(10**2))
# ## Note: a real validation should NEVER be a subset of the training set

# ## Fitting the model to data in batches passed by the user
# recommender = HPF(reindex=False, keep_data=False)
# users_batch1 = np.unique(np.random.randint(10**2, size=20))
# users_batch2 = np.unique(np.random.randint(10**2, size=20))
# users_batch3 = np.unique(np.random.randint(10**2, size=20))
# recommender.partial_fit(counts_df.loc[counts_df.UserId.isin(users_batch1)], nusers=10**2, nitems=10**2)
# recommender.partial_fit(counts_df.loc[counts_df.UserId.isin(users_batch2)])
# recommender.partial_fit(counts_df.loc[counts_df.UserId.isin(users_batch3)])

# ## Making predictions
# # recommender.topN(user=10, n=10, exclude_seen=True) ## not available when using 'partial_fit'
# recommender.topN(user=10, n=10, exclude_seen=False, items_pool=np.array([1,2,3,4]))
# recommender.predict(user=10, item=11)
# recommender.predict(user=[10,10,10], item=[1,2,3])
# recommender.predict(user=[10,11,12], item=[4,5,6])

# ## Evaluating Poisson likelihood
# recommender.eval_llk(counts_df, full_llk=True)

# ## Determining latent factors for a new user, given her item interactions
# nobs_new = 20
# np.random.seed(2)
# counts_df_new = pd.DataFrame({
# 	'ItemId' : np.random.choice(np.arange(nitems), size=nobs_new, replace=False),
# 	'Count' : np.random.gamma(1,1, size=nobs_new).astype('int32')
# 	})
# counts_df_new = counts_df_new.loc[counts_df_new.Count > 0].reset_index(drop=True)
# recommender.predict_factors(counts_df_new)

# ## Adding a user without refitting the whole model
# recommender.add_user(user_id=nusers+1, counts_df=counts_df_new)

# ## Updating data for an existing user without refitting the whole model
# chosen_user = counts_df.UserId.values[10]
# recommender.add_user(user_id=chosen_user, counts_df=counts_df_new, update_existing=True)

**********************************
Hierarchical Poisson Factorization
**********************************

Number of users: 100
Number of items: 100
Latent factors to use: 30

Initializing parameters...
Allocating Phi matrix...
Initializing optimization procedure...
Iteration 10 | train llk: -9872 | train rmse: 1.0955
Iteration 20 | train llk: -9180 | train rmse: 1.0184
Iteration 30 | train llk: -8297 | train rmse: 0.9146
Iteration 40 | train llk: -8342 | train rmse: 0.9285


Optimization finished
Final log-likelihood: -8342
Final RMSE: 0.9285
Minutes taken (optimization part): 0.0

**********************************
Hierarchical Poisson Factorization
**********************************

Number of users: 100
Number of items: 100
Latent factors to use: 30

Initializing parameters...
Allocating Phi matrix...
Initializing optimization procedure...
Iteration 10 | val llk: -99 | val rmse: 1.0556
Iteration 20 | val llk: -91 | val rmse: 0.9649
Iteration 30 | val llk: -90 | val rmse: 0.8848
Iter

True

In [5]:
import pandas as pd, numpy as np
from hpfrec import HPF

train_data = pd.read_csv('C:\\Users\\Sten Stokroos\\Desktop\\zelf\\data\\Webscope_R3\\ydata-ymusic-rating-study-v1_0-train.txt', sep='\t', header=None, names=['UserId', 'ItemId', 'Rating'])

# We don't care about the actual rating value, only if there's been an interaction or not.
# Therefore, we set exposure to 1 for all observed user-item pairs.
train_data['Count'] = 1

# Generate the full user-item matrix and fill in the missing values with 0 (no exposure).
user_item_matrix = train_data.pivot_table(index='UserId', columns='ItemId', values='Count', fill_value=0)

# Now we'll convert the user-item matrix back into the triplet format expected by HPFrec.
# (Note: This step can be memory-intensive and might not be suitable for very large datasets)
user_item_triplets = user_item_matrix.stack().reset_index(name='Count')

# Initialize the HPF model with specified hyperparameters
recommender = HPF(
    k=30, a=0.3, a_prime=0.3, b_prime=0.3,
    c=0.3, c_prime=0.3, d_prime=0.3, ncores=-1,
    stop_crit='train-llk', check_every=10, stop_thr=1e-3,
    maxiter=100, use_float=True, reindex=True, verbose=True,
    random_seed=None, allow_inconsistent_math=False, full_llk=False,
    alloc_full_phi=False, keep_data=True, save_folder=None,
    produce_dicts=True, keep_all_objs=True, sum_exp_trick=False
)

# Fit the HPF model to the binary exposure data
recommender.fit(user_item_triplets)


**********************************
Hierarchical Poisson Factorization
**********************************



c:\Users\Public\ana\Lib\site-packages\hpfrec\__init__.py:469: UserWarning: 'counts_df' contains observations with a count value less than 1, these will be ignored. Any user or item associated exclusively with zero-value observations will be excluded. If using 'reindex=False', make sure that your data still meets the necessary criteria. If you still want to use these observations, set 'stop_crit' to 'diff-norm' or 'maxiter'.
  warnings.warn(
c:\Users\Public\ana\Lib\site-packages\hpfrec\__init__.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.input_df["UserId"], self.user_mapping_ = pd.factorize(self.input_df["UserId"])
c:\Users\Public\ana\Lib\site-packages\hpfrec\__init__.py:479: SettingWithCopyWarning: 
A value is trying to be set on a

Number of users: 15400
Number of items: 1000
Latent factors to use: 30

Initializing parameters...
Allocating Phi matrix...
Initializing optimization procedure...
Iteration 10 | train llk: -1234780 | train rmse: 0.9076
Iteration 20 | train llk: -1044695 | train rmse: 0.8477
Iteration 30 | train llk: -1031548 | train rmse: 0.8439
Iteration 40 | train llk: -1028453 | train rmse: 0.8432
Iteration 50 | train llk: -1026587 | train rmse: 0.8428
Iteration 60 | train llk: -1025480 | train rmse: 0.8427
Iteration 70 | train llk: -1024768 | train rmse: 0.8427


Optimization finished
Final log-likelihood: -1024768
Final RMSE: 0.8427
Minutes taken (optimization part): 0.2



In [6]:
# Calculate the predicted exposures (substitute confounder) for all user-item pairs
predicted_exposures = recommender.predict(user_item_triplets['UserId'], user_item_triplets['ItemId'])

# The result is an array of predicted exposure rates for each user-item pair.
# You can add this back to the dataframe if you want to analyze it further.
user_item_triplets['PredictedExposure'] = predicted_exposures

# If you want to create a full matrix again, you can pivot this data.
# Note that this will create a dense matrix, which may be memory-intensive.
full_predicted_matrix = user_item_triplets.pivot(index='UserId', columns='ItemId', values='PredictedExposure')

# Now you have a full matrix of predicted exposures for each user-item pair.


In [1]:
full_predicted_matrix.iloc[0,13]

# Assuming `user_item_triplets` has a column 'PredictedExposure' with the predicted values
top_25_exposures = user_item_triplets.nsmallest(25, 'PredictedExposure')

# Display the top 25 highest predicted exposures
print(top_25_exposures)


NameError: name 'full_predicted_matrix' is not defined